In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./pqt.csv")
data.head()

,doctor,Gender,Age,st
0,ent,m,50,29
1,paediatrics,m,11,10
2,paediatrics,f,20,14
3,ophthalmology,f,50,11
4,o&g,f,34,27
